In [1]:
import findspark
findspark.init('C:\spark\spark3')
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window
from pyspark.sql import functions as f
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

In [2]:
dataset = spark.read.format('csv').option('header','true').load('loan.csv')

In [3]:
dfOfIncDis = [
dataset.filter(dataset['annual_inc'] < 40000),
dataset.filter(dataset['annual_inc'] >= 40000 ).filter( dataset['annual_inc'] < 60000),
dataset.filter(dataset['annual_inc'] >= 60000 ).filter( dataset['annual_inc'] < 80000),
dataset.filter(dataset['annual_inc'] >= 80000 ).filter( dataset['annual_inc'] < 100000),
dataset.filter(dataset['annual_inc'] >= 100000 )]
report1 = []

data = [["<40k"],["40-60k"],["60-80k"],["80-100k"],[">100k"]]
column  = ['income range']
firstColumn = spark.createDataFrame(data, column)

In [4]:
for df in dfOfIncDis:
    report1.append(df.agg({'loan_amnt': 'avg',
                            'installment': 'avg'}))

In [5]:
merged_df = report1[0].unionByName(report1[1]).unionByName(report1[2]).unionByName(report1[3]).unionByName(report1[4])

In [6]:
merged_df = merged_df.select(col("avg(installment)").alias("avg term"), col("avg(loan_amnt)").alias("avg amount"))

In [7]:
def mergeTwoDF(fst,sec):
    fst = fst.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
    sec = sec.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

    merge = fst.join(sec, fst.row_idx == sec.row_idx).\
             drop("row_idx")
    return merge

In [8]:
report_one = mergeTwoDF(firstColumn,merged_df)

In [9]:
allGrades = dataset.filter(dataset['loan_amnt'] > 1000).select('grade','loan_status')

In [10]:
fullyPaidGrades = dataset.filter(dataset['loan_amnt'] > 1000).filter(dataset['loan_status'] == "Fully Paid").select('grade','loan_status')

In [11]:
allGrades = allGrades.groupBy('grade').count()

In [12]:
allGrades = allGrades.select(col("grade").alias("grade"),col("count").alias("over_all"))

In [13]:
fullyPaidGrades = fullyPaidGrades.groupBy('grade').count()

In [14]:
report_two = mergeTwoDF(fullyPaidGrades,allGrades.select('over_all'))

In [15]:
report_two = report_two.withColumn('percent', (f.col('count')*100)/f.col('over_all'))

In [16]:
report_two = report_two.orderBy('grade', ascending=True).select('grade','percent')

In [17]:
report_one.show()

+------------+-----------------+------------------+
|income range|         avg term|        avg amount|
+------------+-----------------+------------------+
|        <40k|256.3453876457621|  8047.88306162794|
|      40-60k|360.7515654840996|12054.632067154263|
|      60-80k|446.7894502010035|15234.963849034748|
|     80-100k|518.6111203801014|17861.265347404074|
|       >100k|614.1886198497335|21016.407509691584|
+------------+-----------------+------------------+



In [18]:
report_two.show()

+-----+------------------+
|grade|           percent|
+-----+------------------+
|    A|26.712629693467168|
|    B|26.118271567017537|
|    C|21.409498456664107|
|    D|21.503465050842333|
|    E| 18.25747908691337|
|    F|20.497083659789325|
|    G|20.864490242567935|
+-----+------------------+

